In [1]:
import os, sys, time, random
import pandas as pd
import numpy as np
from os.path import join
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn import preprocessing
from sklearn.cross_validation import StratifiedKFold

sys.path.append('/home/ymm/kaggle/xgboost_hyperopt')
import utils.bosch_functions as bosch_functions
from utils.wrapped_xgboost import xgboost_classifier
from utils.validation_tools import score_MCC, MCC, create_validation_index
from utils.models import CombinedModel
from utils.data_munge import remove_single_value_columns
from utils.feature_engineering import NumericalFeatureEngineering, getRelativeTimeColumns, BasicDate_FeatureEngineering
from utils.feature_engineering import getTimeChangeColumns, getTimeSteps, build_IndexFeatures

data_path = '/home/ymm/bosch/'

train_num_file   = 'train_numeric.csv'
train_cat_file   = 'train_categorical.csv'
train_date_file  = 'train_date.csv'
test_num_file    = 'test_numeric.csv'
test_cat_file    = 'test_categorical.csv'
test_date_file   = 'test_date.csv'

sample_submission_file   = 'sample_submission.csv'

start_time_column_name = 'L0_S0_D1'
id_column_name = 'Id'
dep_var_name = 'Response'

In [2]:
#id_df = pd.read_csv(join(data_path, train_num_file),    index_col='Id', usecols=['Id'])
#id_df.shape

In [3]:
tot_row_num = 1183747
num_rows = 150000
skip = sorted(random.sample(xrange(1,tot_row_num + 1),tot_row_num - num_rows))

In [5]:
#'''
start_time = time.time()
## loading the data by using the skipped_row_num list

#train_num = pd.read_csv(join(data_path, train_num_file),    index_col='Id', nrows=num_rows)
#train_dat = pd.read_csv(join(data_path, train_date_file),   index_col='Id', nrows=num_rows)
#train_cat = pd.read_csv(join(data_path, train_cat_file),    index_col='Id', nrows=num_rows)

## randomly select certain rows
train_num = pd.read_csv(join(data_path, train_num_file),    index_col='Id', skiprows=skip, nrows=num_rows)
train_dat = pd.read_csv(join(data_path, train_date_file),   index_col='Id', skiprows=skip, nrows=num_rows)
train_cat = pd.read_csv(join(data_path, train_cat_file),    index_col='Id', skiprows=skip, nrows=num_rows)

test_num = pd.read_csv(join(data_path, test_num_file),      index_col='Id', nrows=num_rows)
test_dat = pd.read_csv(join(data_path, test_date_file),     index_col='Id', nrows=num_rows)
test_cat = pd.read_csv(join(data_path, test_cat_file),      index_col='Id', nrows=num_rows)

print 'finish loading date using {} seconds'.format(round(time.time() - start_time, 0))
#'''

finish loading date using 214.0 seconds


In [6]:
'''
bin_num = 1 ## number of bins to separate data by start_time
tmp_train, tmp_test, bins, bin_names = bosch_functions.create_grouped_index_df(bin_num)
#none_selected_window_num = ['0']
none_selected_window_num = [np.NaN]
skipped_test_row_num = tmp_test.loc[tmp_test['time_window_num'].isin(none_selected_window_num), 'row_num'].tolist()
skipped_train_row_num = tmp_train.loc[tmp_train['time_window_num'].isin(none_selected_window_num), 'row_num'].tolist()


num_rows = 50000
start_time = time.time()
## loading the data by using the skipped_row_num list
train_num = pd.read_csv(join(data_path, train_num_file),    index_col='Id',  skiprows=skipped_train_row_num, nrows=num_rows)
train_dat = pd.read_csv(join(data_path, train_date_file),   index_col='Id',  skiprows=skipped_train_row_num, nrows=num_rows)
test_num = pd.read_csv(join(data_path, test_num_file),      index_col='Id',  skiprows=skipped_test_row_num,  nrows=num_rows)
test_dat = pd.read_csv(join(data_path, test_date_file),     index_col='Id',  skiprows=skipped_test_row_num,  nrows=num_rows)

print 'finish loading date using {} seconds'.format(round(time.time() - start_time, 0))
'''

"\nbin_num = 1 ## number of bins to separate data by start_time\ntmp_train, tmp_test, bins, bin_names = bosch_functions.create_grouped_index_df(bin_num)\n#none_selected_window_num = ['0']\nnone_selected_window_num = [np.NaN]\nskipped_test_row_num = tmp_test.loc[tmp_test['time_window_num'].isin(none_selected_window_num), 'row_num'].tolist()\nskipped_train_row_num = tmp_train.loc[tmp_train['time_window_num'].isin(none_selected_window_num), 'row_num'].tolist()\n\n\nnum_rows = 50000\nstart_time = time.time()\n## loading the data by using the skipped_row_num list\ntrain_num = pd.read_csv(join(data_path, train_num_file),    index_col='Id',  skiprows=skipped_train_row_num, nrows=num_rows)\ntrain_dat = pd.read_csv(join(data_path, train_date_file),   index_col='Id',  skiprows=skipped_train_row_num, nrows=num_rows)\ntest_num = pd.read_csv(join(data_path, test_num_file),      index_col='Id',  skiprows=skipped_test_row_num,  nrows=num_rows)\ntest_dat = pd.read_csv(join(data_path, test_date_file), 

In [7]:
remove_single_value_columns(train_num, 'Response', test=test_num)
remove_single_value_columns(train_dat, test=test_dat)
remove_single_value_columns(train_cat, test=test_cat)

raw train data dimension:  (150000, 969)
raw test data dimension:  (150000, 968)
processed train data dimension:  (150000, 969)
processed test data dimension:  (150000, 968)
raw train data dimension:  (150000, 1156)
raw test data dimension:  (150000, 1156)
processed train data dimension:  (150000, 1146)
processed test data dimension:  (150000, 1146)
raw train data dimension:  (150000, 2140)
raw test data dimension:  (150000, 2140)
processed train data dimension:  (150000, 1730)
processed test data dimension:  (150000, 1730)


#### section to build station-based features

In [8]:
dat_columns = train_dat.columns.tolist()
num_columns = train_num.columns.tolist()
num_columns.remove(dep_var_name)

def build_column_dict(columns):
    col_dict = {}
    for col in columns:
        tmpList = col.split('_')[0:2]
        key = ('_').join(tmpList)
        if key not in col_dict:
            col_dict[key] = [col]
        else:
            col_dict[key].append(col)
            
    return col_dict


def build_station_features(df, col_dict, prefix='dat'):
    features = pd.DataFrame()
    for key, value in col_dict.items():
        features['{}_{}_{}'.format(prefix, key, 'mean')] = df[value].mean(axis=1)
        features['{}_{}_{}'.format(prefix, key, 'max')] = df[value].max(axis=1)
        features['{}_{}_{}'.format(prefix, key, 'min')] = df[value].min(axis=1)
        features['{}_{}_{}'.format(prefix, key, 'var')] = df[value].var(axis=1)
    return features
    
    
dat_col_dict = build_column_dict(dat_columns)
num_col_dict = build_column_dict(num_columns)
        

In [9]:
start_time = time.time()

train_dat_stations = build_station_features(train_dat, dat_col_dict, 'dat')
test_dat_stations = build_station_features(test_dat, dat_col_dict, 'dat')

train_num_stations = build_station_features(train_num, num_col_dict, 'num')
test_num_stations = build_station_features(test_num, num_col_dict, 'num')

print 'finish feature engineering date station using {} minutes'.format(round((time.time() - start_time)/60, 2))

finish feature engineering date station using 0.93 minutes


#### build categorical features

In [10]:
def BasicCat_FeatureEngineering(train_cat):
    ## feature engineering on the date features
    encoder = preprocessing.LabelEncoder()
    column_names = train_cat.columns.tolist()
    column_names.append('NaN')
    encoder.fit(column_names)
    dat_new_fea = pd.DataFrame()
    dat_new_fea['cat_sum'] = train_cat.sum(axis=1)
    dat_new_fea['cat_mean'] = train_cat.mean(axis=1)
    dat_new_fea['cat_nan_count'] = train_cat.isnull().sum(axis=1)
    dat_new_fea['cat_max'] = train_cat.max(axis=1)
    dat_new_fea['cat_min'] = train_cat.min(axis=1)
    dat_new_fea['cat_max_min_diff'] = dat_new_fea['cat_max'] - dat_new_fea['cat_min']
    dat_new_fea['cat_max_min_ratio'] = dat_new_fea['cat_min'] / dat_new_fea['cat_max']

    dat_new_fea['cat_idxmax'] = train_cat.idxmax(axis=1)
    dat_new_fea['cat_idxmax'].fillna('NaN', inplace=True)
    dat_new_fea['cat_idxmax'] = encoder.transform(dat_new_fea['cat_idxmax'])
    dat_new_fea['cat_idxmin'] = train_cat.idxmin(axis=1)
    dat_new_fea['cat_idxmin'].fillna('NaN', inplace=True)
    dat_new_fea['cat_idxmin'] = encoder.transform(dat_new_fea['cat_idxmin'])
    return dat_new_fea



def encode_categorical_by_dep_var(train, test, dep_var_column='Response'):
    for col_name in train.columns:
        if col_name == dep_var_column:
            continue
        dep_var_mean = train[[col_name, dep_var_column]].groupby(col_name).mean()
    
        dep_var_dict = {}
        for level in dep_var_mean.index.tolist():
            dep_var_dict[level] = dep_var_mean.ix[level, dep_var_column]
    
        train[col_name] = train[col_name].replace(dep_var_dict)  
        test[col_name] = test[col_name].replace(dep_var_dict)  

In [11]:
start_time = time.time()

train_cat['Response'] = train_num['Response']
encode_categorical_by_dep_var(train_cat, test_cat)
train_cat.drop('Response', axis=1, inplace=True)

train_cat_Basics = BasicCat_FeatureEngineering(train_cat)
test_cat_Basics  = BasicCat_FeatureEngineering(train_cat)

print 'finish generating categorical features using {} seconds'.format(round(time.time() - start_time, 0))


finish generating categorical features using 93.0 seconds


In [12]:
print train_cat.shape, test_cat.shape
train_cat.head()

(150000, 1730) (150000, 1730)


,L0_S1_F25,L0_S1_F27,L0_S1_F29,L0_S1_F31,L0_S2_F33,L0_S2_F35,L0_S2_F37,L0_S2_F39,L0_S2_F41,L0_S2_F43,...,L3_S49_F4225,L3_S49_F4227,L3_S49_F4229,L3_S49_F4230,L3_S49_F4232,L3_S49_F4234,L3_S49_F4235,L3_S49_F4237,L3_S49_F4239,L3_S49_F4240
Id,,,,,,,,,,,,,,,,,,,,,
38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
start_time = time.time()
combined_train_cat = pd.concat([train_cat, train_cat_Basics], axis=1)
combined_test_cat  = pd.concat([test_cat, test_cat_Basics], axis=1)                                                                                                                                                 
print 'finish feature engineering date using {} seconds'.format(round((time.time() - start_time), 2))

finish feature engineering date using 2.64 seconds


In [ ]:
##### numerical feature engineering

In [14]:
#### numerical feature engineering work
train_num_Basics = NumericalFeatureEngineering(train_num)
test_num_Basics = NumericalFeatureEngineering(test_num)

combined_train_num = pd.concat([train_num, train_num_Basics], axis=1)
combined_test_num  = pd.concat([test_num, test_num_Basics], axis=1)                                                                            
print 'combined train numerical feature shape: {}, combined test numerical features shape: {}'.format(combined_train_num.shape, combined_test_num.shape)

combined train numerical feature shape: (150000, 978), combined test numerical features shape: (150000, 977)


In [ ]:
#### section of date features

In [15]:
## basic features from tmp_train_dat
train_dat_Basics = BasicDate_FeatureEngineering(train_dat)
test_dat_Basics  = BasicDate_FeatureEngineering(test_dat)

In [16]:
## normalized date columns
train_dat_Norm = train_dat.apply(getRelativeTimeColumns, axis=1)
test_dat_Norm  = test_dat.apply(getRelativeTimeColumns, axis=1)
## remove single-valued columns
remove_single_value_columns(train_dat_Norm, test=test_dat_Norm)

raw train data dimension:  (150000, 1146)
raw test data dimension:  (150000, 1146)
processed train data dimension:  (150000, 969)
processed test data dimension:  (150000, 969)


In [46]:
encoder = preprocessing.LabelEncoder()
column_names = train_dat.columns.tolist()
column_names.append('NaN')
encoder.fit(column_names)

LabelEncoder()

In [47]:
## TimeDiff features
start_time = time.time()
train_dat_TimeDiff = train_dat.apply(getTimeChangeColumns, axis=1)
test_dat_TimeDiff  = test_dat.apply(getTimeChangeColumns, axis=1)
TimeDiff_ColumnNames = ['time_diff_start_col', 'time_diff_end_col', 'time_diff_value',
                        'time_ratio_value', 'first_time_value', 'last_time_value', 'first_date_value']
train_dat_TimeDiff.columns = TimeDiff_ColumnNames
test_dat_TimeDiff.columns = TimeDiff_ColumnNames

for column in ['time_diff_start_col', 'time_diff_end_col']:
    train_dat_TimeDiff[column].fillna('NaN', inplace=True)
    train_dat_TimeDiff[column] = encoder.transform(train_dat_TimeDiff[column])
    
    test_dat_TimeDiff[column].fillna('NaN', inplace=True)
    test_dat_TimeDiff[column] = encoder.transform(test_dat_TimeDiff[column])
    
print 'finish feature TimeDiff engineering date using {} minutes'.format(round((time.time() - start_time)/60, 2))

In [48]:
start_time = time.time()

## section to create timeStep features

unique_value_counts = 6
timeStep_columnNames = []
column_name_columns = []
for i in xrange(unique_value_counts):
    timeStep_columnNames.extend(['time_diff_step_{}'.format(i), 'column_counts_step_{}'.format(i),
                                 'time_cost_step_{}'.format(i), 'first_column_step_{}'.format(i)])
    column_name_columns.append('first_column_step_{}'.format(i))

train_dat_TimeStep = train_dat_Norm.apply(getTimeSteps, axis=1)
test_dat_TimeStep  = test_dat_Norm.apply(getTimeSteps, axis=1)
train_dat_TimeStep.columns = timeStep_columnNames
test_dat_TimeStep.columns  = timeStep_columnNames

for column in column_name_columns:
    train_dat_TimeStep[column].fillna('NaN', inplace=True)
    test_dat_TimeStep[column].fillna('NaN', inplace=True)
    train_dat_TimeStep[column] = encoder.transform(train_dat_TimeStep[column])
    test_dat_TimeStep[column] = encoder.transform(test_dat_TimeStep[column])


print 'finish generating TimeStep features using {} seconds'.format(round(time.time() - start_time, 0))


finish generating TimeStep features using 1876.0 seconds


In [49]:
combined_train_dat = pd.concat([train_dat_Norm, train_dat_Basics, train_dat_TimeDiff, train_dat_TimeStep], axis=1)
combined_test_dat  = pd.concat([test_dat_Norm, test_dat_Basics, test_dat_TimeDiff, test_dat_TimeStep], axis=1)

finish feature engineering date using 0.03 minutes


In [50]:
print combined_train_dat.shape, combined_test_dat.shape
combined_train_dat.head()

(150000, 1010) (150000, 1010)


,L0_S0_D1,L0_S0_D3,L0_S0_D5,L0_S0_D7,L0_S0_D9,L0_S0_D11,L0_S0_D13,L0_S0_D15,L0_S0_D17,L0_S0_D19,...,time_cost_step_3,first_column_step_3,time_diff_step_4,column_counts_step_4,time_cost_step_4,first_column_step_4,time_diff_step_5,column_counts_step_5,time_cost_step_5,first_column_step_5
Id,,,,,,,,,,,,,,,,,,,,,
38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.088621,1018,NaN,0,0.000000,1146,NaN,0,0.0,1146
71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.376667,1049,NaN,0,0.000000,1146,NaN,0,0.0,1146
73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,1146,NaN,0,0.000000,1146,NaN,0,0.0,1146
75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.108000,1018,1.64,14,0.117143,1041,NaN,0,0.0,1146
93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.100667,1018,1.52,14,0.108571,1041,NaN,0,0.0,1146


In [51]:
train_test_datIndex_features = build_IndexFeatures(combined_train_dat, combined_test_dat)

#### load the feature importances from xgboost models

In [52]:
xgb_feature_imp = pd.read_csv('/home/ymm/full_data_xgb_feature_importance.csv', index_col='feature')

In [53]:
xgb_feature_imp.head()

,fscore_0,norm_fscore_0,fscore_1,norm_fscore_1,fscore_2,norm_fscore_2,fscore_3,norm_fscore_3,fscore_4,norm_fscore_4,...,fscore_8,norm_fscore_8,fscore_9,norm_fscore_9,fscore_10,norm_fscore_10,fscore_11,norm_fscore_11,fscore_sum,norm_fscore_sum
feature,,,,,,,,,,,,,,,,,,,,,
first_time_value_index_diff_1,1357.0,0.013342,1480.0,0.014495,773.0,0.032962,1701.0,0.011999,1198.0,0.018810,...,1723.0,0.012308,747.0,0.031607,1671.0,0.011886,1152.0,0.018246,15033.0,0.228153
L3_S33_F3857,1317.0,0.012948,1226.0,0.012007,316.0,0.013475,1797.0,0.012676,648.0,0.010175,...,1745.0,0.012465,391.0,0.016544,1874.0,0.013330,708.0,0.011214,12276.0,0.149472
last_time_value_index_diff_1,1015.0,0.009979,1059.0,0.010372,410.0,0.017483,1310.0,0.009241,840.0,0.013189,...,1488.0,0.010629,382.0,0.016163,1329.0,0.009453,743.0,0.011768,10737.0,0.145426
L3_S33_F3859,1223.0,0.012024,1161.0,0.011371,264.0,0.011258,1722.0,0.012147,744.0,0.011682,...,1556.0,0.011115,375.0,0.015867,1649.0,0.011729,786.0,0.012449,11676.0,0.144343
last_time_value_index_diff_0,1020.0,0.010028,1087.0,0.010646,350.0,0.014925,1351.0,0.009530,707.0,0.011101,...,1391.0,0.009937,310.0,0.013117,1320.0,0.009389,612.0,0.009693,10258.0,0.132143


In [54]:
## sort by the norm_fscore_sum
sorted_combined_imp = xgb_feature_imp.sort_values(by=['norm_fscore_sum'], ascending=False)
imp_feature = sorted_combined_imp.index[sorted_combined_imp['norm_fscore_sum'] >= 0.005].tolist()

In [55]:
def select_important_features(df, imp_feature, test_df = None, dep_var_name = 'Response'):
    imp_col_names = [col for col in df.columns if col in imp_feature]
    print 'total {} columns in original DataFrame, select {} columns'.format(df.shape[1], len(imp_col_names))
    train_col_names = imp_col_names[:]
    test_col_names = imp_col_names[:]
    if dep_var_name in df.columns:    
        train_col_names.append(dep_var_name)
    if test_df is None:
        return df[train_col_names]
    else:
        return df[train_col_names], test_df[test_col_names]


In [56]:
selected_train_num, selected_test_num  = select_important_features(combined_train_num, imp_feature, combined_test_num)
selected_train_dat, selected_test_dat  = select_important_features(combined_train_dat, imp_feature, combined_test_dat)
selected_train_idx = select_important_features(train_test_datIndex_features, imp_feature)

total 978 columns in original DataFrame, select 350 columns
total 1010 columns in original DataFrame, select 49 columns
total 23 columns in original DataFrame, select 22 columns


In [57]:
print selected_train_num.shape, selected_test_num.shape

(150000, 351) (150000, 350)


In [58]:
## combined data with categorical features
combined_train = pd.concat([selected_train_num, selected_train_dat, selected_train_idx.ix[combined_train_num.index, :]], axis=1)
combined_test  = pd.concat([selected_test_num,  selected_test_dat,  selected_train_idx.ix[combined_test_num.index, :]], axis=1)

#### combine all the features together

In [59]:
#combined_train = pd.concat([train_dat_stations, train_num_stations, combined_train_num, combined_train_dat, train_test_datIndex_features.ix[combined_train_num.index, :]], axis=1)
#combined_test  = pd.concat([test_dat_stations, test_num_stations, combined_test_num,  combined_test_dat,  train_test_datIndex_features.ix[combined_test_num.index, :]], axis=1)

In [60]:
#combined_train = pd.concat([combined_train_num, combined_train_dat, train_test_datIndex_features.ix[combined_train_num.index, :]], axis=1)
#combined_test  = pd.concat([combined_test_num,  combined_test_dat,  train_test_datIndex_features.ix[combined_test_num.index, :]], axis=1)

In [61]:
## combined data with categorical features
#combined_train = pd.concat([combined_train_cat, combined_train_num, combined_train_dat, train_test_datIndex_features.ix[combined_train_num.index, :]], axis=1)
#combined_test  = pd.concat([combined_test_cat,  combined_test_num,  combined_test_dat,  train_test_datIndex_features.ix[combined_test_num.index, :]], axis=1)

In [62]:
print combined_test.shape
combined_test.head()

(150000, 421)


,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,L0_S0_F18,...,first_date_value_index_ratio_1,first_date_value_index_ratio_2,time_ratio_value_index_diff_0,time_ratio_value_index_diff_1,first_time_value_index_diff_0,first_time_value_index_diff_1,last_time_value_index_diff_0,last_time_value_index_diff_1,first_date_value_index_diff_0,first_date_value_index_diff_1
Id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,219053.000000,99722.000,-178143,-3232,-644790,-3232,-138624,-3232,-2363612,-1
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,69950.000000,47824.000,-208741,-62612,-2133092,-5943,-2205351,-61348,1,-1
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,71142.333333,31963.000,-2353451,-24124,-2239878,-24124,-2271710,-24124,1,-5
5,-0.016,-0.026,-0.033,-0.016,0.205,-0.157,0.0,0.008,0.087,0.077,...,43210.200000,22891.200,-1753148,-140294,-291851,-140294,-605452,-4697,-221062,-63560
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6794.500000,14364.625,-1903006,-565415,-1497987,-565415,-403250,-4313,5,-9


In [63]:
print combined_train.shape
combined_train.head()

(150000, 422)


,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,L0_S0_F18,...,first_date_value_index_ratio_1,first_date_value_index_ratio_2,time_ratio_value_index_diff_0,time_ratio_value_index_diff_1,first_time_value_index_diff_0,first_time_value_index_diff_1,last_time_value_index_diff_0,last_time_value_index_diff_1,first_date_value_index_diff_0,first_date_value_index_diff_1
Id,,,,,,,,,,,,,,,,,,,,,
38,0.252,0.250,0.003,-0.016,0.118,-0.294,-0.044,-0.232,-0.179,0.083,...,1113.421053,7471.368421,-154980,-70720,-2267819,-70720,-2286269,-3883,-2290079,-70720
71,-0.167,-0.168,0.276,0.330,0.074,0.161,0.052,0.248,0.163,0.120,...,3731.732394,3904.549296,-298727,-158106,-2322164,-26319,-230138,-4153,-2322164,-4153
73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3740.808219,3878.397260,-1847589,-4305,-2236941,-5381,-2236941,-1956890,6,-7
75,0.036,0.056,0.312,0.294,-0.056,-0.066,0.008,0.048,-0.021,0.083,...,3641.066667,2359.453333,-569347,-177685,-293991,-58790,-273753,-126148,-273753,-58790
93,-0.029,-0.071,-0.015,0.003,0.031,0.116,0.008,0.088,0.071,0.083,...,217.967742,44.150538,-370576,-60160,-254637,-4106,-1546983,-57270,-546624,-60160


In [64]:
params = {}
params["eta"]                      = 0.0075
params["subsample"]                = 0.8
params["colsample_bytree"]         = 0.8
params["num_round"]                = 501
params["max_depth"]                = 5
params["gamma"]                    = 0
params["metrics"]                  = 'auc'
params['eval_metric']              = 'auc'
params["seed"]                     = 999
params['verbose_eval']             = 50
## whether to use weights
params['use_base_score']           = True
params['use_weights']              = True
#params['use_scale_pos_weight']     = True
params["val"]                      = False


In [71]:

skf = StratifiedKFold(combined_train[dep_var_name], 2)

for train_index, valid_index in skf:
    valid_data = combined_train.iloc[valid_index]
    tmp_train  = combined_train.iloc[train_index]

    y = tmp_train[dep_var_name].values
    X = tmp_train.drop(dep_var_name, axis=1)

    valid_y = valid_data[dep_var_name].values
    valid_X = valid_data.drop(dep_var_name, axis=1)
    
    model = xgboost_classifier(label_name = dep_var_name, params = params, model_file='test_bosch_xgb_model')
    model.fit(tmp_train, dep_var_name)
    
    pred = model.predict(valid_X)
    print 'result from using constant fraction: \n', score_MCC(valid_y, pred)
    print '\n'
    print 'result from using flexsible threshold:', CombinedModel.mcc_eval_func(valid_y, pred)

scale_pos_weight: 168.300225734
a base_score 0.00590666666667 is used in the xgboost model...



####################
 train the xgboost without early stopping
####################


[0]	train-auc:0.672788
[50]	train-auc:0.801495
[100]	train-auc:0.812508
[150]	train-auc:0.834507
[200]	train-auc:0.871128
[250]	train-auc:0.902938
[300]	train-auc:0.925638
[350]	train-auc:0.941249
[400]	train-auc:0.953326
[450]	train-auc:0.963176
[500]	train-auc:0.969778
the xgboost fit is finished by using 44.9011330605 seconds, saved into test_bosch_xgb_model
in the prediction step, dep_var_name is not provided....
result from using constant fraction: 
mean of groud truth: 0.00590666666667
threshold for preds: 0.0360628595992
0.177988898218


result from using flexsible threshold: (0.19912017578559876, 0.07214768975973129)
scale_pos_weight: 168.300225734
a base_score 0.00590666666667 is used in the xgboost model...



####################
 train the xgboost without early stopping
####################


[0]	train-auc:0.704563
[50]	train-auc:0.843982
[100]	train-auc:0.856223
[150]	train-auc:0.886661
[200]	train-auc:0.910756
[250]	train-auc:0.927459
[300]	train-auc:0.941276
[350]	train-auc:0.950944
[400]	train-auc:0.95841
[450]	train-auc:0.965689
[500]	train-auc:0.970688
the xgboost fit is finished by using 44.3524279594 seconds, saved into test_bosch_xgb_model
in the prediction step, dep_var_name is not provided....
result from using constant fraction: 
mean of groud truth: 0.00590666666667
threshold for preds: 0.0398607837876
0.187071893818


result from using flexsible threshold: (0.2160595638882123, 0.07732672244310379)


In [66]:
############## Section of regular validation #######################
train_index, valid_index = create_validation_index(combined_train, 0.3, dep_var_name, True)
valid_data = combined_train.ix[valid_index]
tmp_train  = combined_train.ix[train_index]

y = tmp_train[dep_var_name].values
X = tmp_train.drop(dep_var_name, axis=1)

valid_y = valid_data[dep_var_name].values
valid_X = valid_data.drop(dep_var_name, axis=1)

In [67]:
params = {}
params["eta"]                      = 0.0075
params["subsample"]                = 0.8
params["colsample_bytree"]         = 0.8
params["num_round"]                = 501
params["max_depth"]                = 5
params["gamma"]                    = 0
params["metrics"]                  = 'auc'
params['eval_metric']              = 'auc'
params["seed"]                     = 999
params['verbose_eval']             = 50
## whether to use weights
params['use_base_score']           = True
params['use_weights']              = True
#params['use_scale_pos_weight']     = True
params["val"]                      = False

model = xgboost_classifier(label_name = dep_var_name, params = params, model_file='bosch_xgb_model')
model.fit(tmp_train, dep_var_name)

scale_pos_weight: 168.08373591
a base_score 0.00591422938829 is used in the xgboost model...



####################
 train the xgboost without early stopping
####################


[0]	train-auc:0.620821
[50]	train-auc:0.781388
[100]	train-auc:0.802672
[150]	train-auc:0.825852
[200]	train-auc:0.855809
[250]	train-auc:0.882926
[300]	train-auc:0.906002
[350]	train-auc:0.922022
[400]	train-auc:0.933902
[450]	train-auc:0.944958
[500]	train-auc:0.95228
the xgboost fit is finished by using 62.2322840691 seconds, saved into bosch_xgb_model


In [68]:
pred = model.predict(valid_X)

print 'result from using constant fraction: \n', score_MCC(valid_y, pred)
print '\n'
print 'result from using flexsible threshold:', CombinedModel.mcc_eval_func(valid_y, pred)

in the prediction step, dep_var_name is not provided....
result from using constant fraction: 
mean of groud truth: 0.00588901975599
threshold for preds: 0.0435452772094
0.218036510999


result from using flexsible threshold: (0.25054412667143927, 0.12990249693393707)
